# MATRICE DI CORRELAZIONE

Come prima operazione, importiamo le librerie necessarie

In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import pandas as pd
import numpy as np

Successivamente si procede con l'importazione del dataset contenuto nella cartella "data". Per raggiungere tale scopo si utilizza il metodo **read_excel()** del modulo **pandas**

In [2]:
table = pd.read_excel('https://github.com/collab-uniba/PanelJamDataExtractor/blob/master/data/TabellaCompleta.xlsx?raw=true', index = False)

Una volta importato il dataset si devono eliminare alcune feature contenute in esso, questo perchè:

-le feature **id_prog**,**id_panel**,**panel_author** contengono delle stringhe che indicano rispettivamente, l'id dei progetti estratti, l'id dei panel contenuti in ogni progetto e il nome del creatore di ogni panel. Queste feature non sono rilevanti per il calcolo della matrice di correlazione(mostrata successivamente) e nemmeno per la costruzione del modello di regressione.

-le feature **Likes**, **Views**, **Comments** e **time** fanno riferimento ai progetti e non ai singoli panel, per questo motivo non vengono considerati.

Si procede con la costruzione della matrice di correlazione con lo scopo di osservare eventuali problemi di **multicollinearità** tra le feature. Se il grado di correlazione tra due feature è maggiore del limite consentito di **0.7**, se ne deve eliminare una delle due. La matrice verrà costruita utilizzando il metodo **corr()** del modulo **pandas**

In [3]:
#time = table['time'].tolist()
#table['time'] = list(map(lambda sub:int(''.join([ele for ele in sub if ele.isnumeric()])), time))
table = table.drop(columns = ['id_prog','id_panel','Likes','Views','Comments','panel_author','time','author_stars'])

corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,project_depth,extended,has_avatar_author,has_bio_author,author_followers,tot_loves_author,tot_views_author,author_ranking,panel_stars
project_depth,1.00,-0.42,0.06,0.03,0.02,0.04,0.05,0.07,0.07
extended,-0.42,1.00,-0.03,-0.03,-0.02,-0.03,-0.03,-0.04,-0.17
has_avatar_author,0.06,-0.03,1.00,0.24,0.18,0.15,0.11,0.09,0.08
has_bio_author,0.03,-0.03,0.24,1.00,0.22,0.12,0.01,-0.03,0.06
author_followers,0.02,-0.02,0.18,0.22,1.00,0.71,0.63,0.33,0.18
tot_loves_author,0.04,-0.03,0.15,0.12,0.71,1.00,0.76,0.31,0.19
tot_views_author,0.05,-0.03,0.11,0.01,0.63,0.76,1.00,0.63,0.12
author_ranking,0.07,-0.04,0.09,-0.03,0.33,0.31,0.63,1.00,0.12
panel_stars,0.07,-0.17,0.08,0.06,0.18,0.19,0.12,0.12,1.00


Come si può osservare dalla matrice, c'è un alto grado di correlazione tra **tot_loves_author** e **author_stars** (**0.76**) superiore al limite di **0.7**. Questo è dovuto dalla presenza di entrambe le feature nel calcolo del ranking d'autore, per questo si decide di eliminare tot_views_author. 

In [4]:
table = table.drop(columns = ['tot_views_author'])
corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,project_depth,extended,has_avatar_author,has_bio_author,author_followers,tot_loves_author,author_ranking,panel_stars
project_depth,1.00,-0.42,0.06,0.03,0.02,0.04,0.07,0.07
extended,-0.42,1.00,-0.03,-0.03,-0.02,-0.03,-0.04,-0.17
has_avatar_author,0.06,-0.03,1.00,0.24,0.18,0.15,0.09,0.08
has_bio_author,0.03,-0.03,0.24,1.00,0.22,0.12,-0.03,0.06
author_followers,0.02,-0.02,0.18,0.22,1.00,0.71,0.33,0.18
tot_loves_author,0.04,-0.03,0.15,0.12,0.71,1.00,0.31,0.19
author_ranking,0.07,-0.04,0.09,-0.03,0.33,0.31,1.00,0.12
panel_stars,0.07,-0.17,0.08,0.06,0.18,0.19,0.12,1.00


# MODELLO DI REGRESSIONE

Dopo aver costruito la matrice di correlazione ed eliminato eventuali problemi di collinearità, si procede con la costruzione del modello di regressione. Il primo passaggio consiste nel dividere il dataset in due, dove il primo conterrà solo le feature non numeriche ed è stato etichettato con il nome **logit**, il secondo conterrà solo le feature numeriche ed è stato etichettato con il nome **numerical**. In seguito si spiegherà il motivo di questa divisione.

In [5]:
featureTable = table
logit = featureTable.drop(columns = ['project_depth','author_followers','tot_loves_author',
                'author_ranking','panel_stars'])
numerical = featureTable.drop(columns = ['extended','has_avatar_author','has_bio_author'])

bisogna effettuare una log-normalizzazione delle feature numeriche in quanto alcune di esse contengono valori troppo grandi. Dopo di chè si procede alla ricostruzione della tabella, andando ad unire la tabella numerica(ottenuta dalla log-normalizzazione) alla tabella non numerica.

In [6]:
normTable = np.log1p(numerical)
normTable['extended'] = logit['extended']
normTable['has_avatar_author'] = logit['has_avatar_author']
normTable['has_bio_author'] = logit['has_bio_author']

Per costruire il modello di regressione si è utilizzata la funzione **glm** del package **statsmodel**, alla quale sono stati passati in input 3 parametri.

-**formula**: prima del simbolo tilde (~) si deve specificare la variabile dipendente, mentre dopo bisogna specificare le variabili indipendenti, i quali costituiranno i predittori del modello.

-**data**: la tabella da cui estrarre i valori delle variabili indipendenti.

-**family**: specifica il modello di regressione che si vuole andare a costruire, in questo caso binomial ed è specificato da **sm.families.Binomial()**.

In [7]:
formula = """extended ~ project_depth + author_followers + tot_loves_author +
             has_bio_author + has_avatar_author + author_ranking + panel_stars"""

model = smf.glm(formula = formula, data = normTable, family=sm.families.Binomial(statsmodels.genmod.families.links.logit)).fit()
print(model.summary())

                           Generalized Linear Model Regression Results                           
Dep. Variable:     ['extended[False]', 'extended[True]']   No. Observations:                13748
Model:                                               GLM   Df Residuals:                    13740
Model Family:                                   Binomial   Df Model:                            7
Link Function:                                     logit   Scale:                          1.0000
Method:                                             IRLS   Log-Likelihood:                -5333.8
Date:                                   Thu, 04 Jun 2020   Deviance:                       10668.
Time:                                           16:59:24   Pearson chi2:                 1.60e+04
No. Iterations:                                        6                                         
Covariance Type:                               nonrobust                                         
                    

d:\dev-cpp\anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  after removing the cwd from sys.path.


In [8]:
print(np.exp(model.params))

Intercept                     0.001344
has_bio_author[T.True]        1.086741
has_avatar_author[T.True]     0.893892
project_depth                49.131351
author_followers              0.965896
tot_loves_author              1.023486
author_ranking                0.899986
panel_stars                   1.839916
dtype: float64
